In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import sklearn
from sklearn.model_selection import train_test_split

import itertools
from typing import List, Tuple
import copy
import seaborn as sns
import unicodedata
import re
import numpy as np
import os
import io
from skimage.io import imread
import time
from ast import literal_eval
import math
from collections import defaultdict
from statistics import median, mean

import keras
from keras import optimizers
from keras.preprocessing.image import load_img
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Parse and Load Dataset

In [2]:
TARGET_SIZE = (640, 360)
IMAGE_EXAMPLE_SHAPE = TARGET_SIZE + (3,)

LANDMARK_EXAMPLE_SHAPE = (39, )

BASE_PATH = '/home/lancecotingkeh/aggregated/'
LABEL_TO_ENCODING = {'proper': 0, 'crooked': 1, 'lockout': 2, 'rounded': 3, 'squat': 4}
NUM_CLASSES = len(LABEL_TO_ENCODING)

In [3]:
df_metadata = pd.read_csv('/home/lancecotingkeh/aggregated/pose_landmarks.csv')

In [4]:
df_metadata.head(3)

,relative_image_path,data_split,label,landmark_nose,landmark_left_shoulder,landmark_right_shoulder,landmark_left_elbow,landmark_right_elbow,landmark_left_wrist,landmark_right_wrist,landmark_left_hip,landmark_right_hip,landmark_left_knee,landmark_right_knee,landmark_left_ankle,landmark_right_ankle
0,5_lockout/lyron_deg135_take2/frame46.jpg,train,proper,"(0.8408212661743164, 0.44535547494888306, -0.6...","(0.7523922920227051, 0.483453631401062, -0.080...","(0.59047532081604, 0.4363204836845398, -0.5876...","(0.7419312596321106, 0.6247938871383667, 0.025...","(0.5161336064338684, 0.5835246443748474, -0.57...","(0.7500355839729309, 0.7468990683555603, -0.06...","(0.5082715749740601, 0.7348905205726624, -0.60...","(0.4615359604358673, 0.5611119866371155, 0.148...","(0.34500354528427124, 0.5380272269248962, -0.1...","(0.701583981513977, 0.6245787739753723, -0.039...","(0.5147499442100525, 0.576558530330658, -0.833...","(0.6168646812438965, 0.8138770461082458, 0.163...","(0.4869398772716522, 0.7884341478347778, -0.77..."
1,5_lockout/lyron_deg135_take2/frame203.jpg,train,proper,"(0.8654815554618835, 0.4472369849681854, -0.59...","(0.7866912484169006, 0.5048587322235107, -0.12...","(0.6134108901023865, 0.4505915343761444, -0.55...","(0.7522295713424683, 0.6383607983589172, -0.09...","(0.5366990566253662, 0.5983656644821167, -0.55...","(0.7443258762359619, 0.7658225893974304, -0.16...","(0.5270508527755737, 0.7469443678855896, -0.51...","(0.49029654264450073, 0.565066397190094, 0.141...","(0.3752322196960449, 0.5404682159423828, -0.14...","(0.721193253993988, 0.6225733160972595, 0.0097...","(0.604214072227478, 0.5913941860198975, -0.544...","(0.6175094246864319, 0.7968677282333374, 0.155...","(0.4880143404006958, 0.7921530604362488, -0.41..."
2,5_lockout/lyron_deg135_take2/frame207.jpg,train,proper,"(0.8618521690368652, 0.42703595757484436, -0.5...","(0.7759232521057129, 0.48765045404434204, -0.0...","(0.6108279228210449, 0.43047401309013367, -0.4...","(0.7476211786270142, 0.6245181560516357, 0.032...","(0.526120126247406, 0.5700463652610779, -0.489...","(0.7355574369430542, 0.7427244186401367, -0.06...","(0.5221640467643738, 0.7184542417526245, -0.49...","(0.4879094958305359, 0.5496541261672974, 0.132...","(0.37393078207969666, 0.5203757286071777, -0.1...","(0.7107850313186646, 0.6180444359779358, 0.138...","(0.574447512626648, 0.5620325207710266, -0.697...","(0.5569970607757568, 0.7731618285179138, 0.340...","(0.47665083408355713, 0.7690679430961609, -0.6..."


In [5]:
landmark_columns = [col for col in df_metadata if col.startswith('landmark_')]
landmark_columns

['landmark_nose',
 'landmark_left_shoulder',
 'landmark_right_shoulder',
 'landmark_left_elbow',
 'landmark_right_elbow',
 'landmark_left_wrist',
 'landmark_right_wrist',
 'landmark_left_hip',
 'landmark_right_hip',
 'landmark_left_knee',
 'landmark_right_knee',
 'landmark_left_ankle',
 'landmark_right_ankle']

In [6]:
def load_preprocess_imgs(image_paths: List[str]) -> np.ndarray:
    images = []
    for image_path in image_paths:
        img = load_img(image_path, target_size=TARGET_SIZE)        
        np_img = np.asarray(img)

        assert np_img.shape == IMAGE_EXAMPLE_SHAPE
        
        images.append(np_img)
    return np.array(images)

In [7]:
def load_preprocess_landmarks(df_metadata) -> np.ndarray:
    landmark_examples = []
    
    for row in df_metadata.iterrows():
        single_example = []
        
        for landmark in landmark_columns:
            x, y, z = literal_eval(row[1][landmark])
            
            single_example.extend([x, y, z])
        
        landmark_examples.append(np.array(single_example))
    return np.array(landmark_examples)

In [8]:
# load training data

df_train = df_metadata[df_metadata['data_split'] == 'train']
X_train_imgs = load_preprocess_imgs([f'{BASE_PATH}{p}' for p in df_train['relative_image_path']])
X_train_landmarks = load_preprocess_landmarks(df_train)
Y_train = keras.utils.to_categorical([LABEL_TO_ENCODING[l] for l in df_train['label']], 5)

In [9]:
len(df_train), len(X_train_imgs), len(X_train_landmarks), len(Y_train)

(589, 589, 589, 589)

In [10]:
# load validation data
df_val = df_metadata[df_metadata['data_split'] == 'val']
X_val_imgs = load_preprocess_imgs([f'{BASE_PATH}{p}' for p in df_val['relative_image_path']])
X_val_landmarks = load_preprocess_landmarks(df_val)
Y_val = keras.utils.to_categorical([LABEL_TO_ENCODING[l] for l in df_val['label']], 5)

In [11]:
len(df_val), len(X_val_imgs), len(X_val_landmarks), len(Y_val)

(100, 100, 100, 100)

In [12]:
# load test data
df_test = df_metadata[df_metadata['data_split'] == 'test']
X_test_imgs = load_preprocess_imgs([f'{BASE_PATH}{p}' for p in df_test['relative_image_path']])
X_test_landmarks = load_preprocess_landmarks(df_test)
Y_test = keras.utils.to_categorical([LABEL_TO_ENCODING[l] for l in df_test['label']], 5)

In [13]:
len(df_test), len(X_test_imgs), len(X_test_landmarks), len(Y_test)

(200, 200, 200, 200)

In [14]:
IMAGE_EXAMPLE_SHAPE

(640, 360, 3)

# Define Layers

In [15]:
initializer = tf.initializers.VarianceScaling(scale=2.0)

class ConvGroupLayer(tf.keras.layers.Layer):
    def __init__(self, channel_size: int, filter_size: Tuple[int, int], dropout_rate: float):
        self.conv = Conv2D(channel_size, filter_size, padding='same', kernel_initializer=initializer)
        self.max_pool = MaxPooling2D(filter_size)
        self.bn = BatchNormalization(axis=-1)
        self.relu = LeakyReLU(alpha=0.1)
        self.drop_out = Dropout(dropout_rate)
    
    def __call__(self, x):
        x = self.conv(x)
        x = self.max_pool(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.drop_out(x)
        
        return x

class CNNTower(tf.keras.layers.Layer):
    def __init__(self, num_conv_layers:int, initial_channel_size: int, initial_filter_size: Tuple[int, int], dropout_rate: float):
        self.initial_conv = Conv2D(initial_channel_size, initial_filter_size, padding='same', kernel_initializer=initializer, input_shape=IMAGE_EXAMPLE_SHAPE)
        self.initial_max_pool = MaxPooling2D(initial_filter_size, padding='same')
        self.initial_bn = BatchNormalization(axis=-1)
        self.initial_relu = LeakyReLU(alpha=0.1)
        self.initial_drop_out = Dropout(dropout_rate)
        
        self.intermediate_conv_layers = []
        
        for idx in range(num_conv_layers):
            channel_size = initial_channel_size//(2**(idx+1))
            
            self.intermediate_conv_layers.append(ConvGroupLayer(channel_size, initial_filter_size, dropout_rate))
            
    def __call__(self, x):
        x = self.initial_conv(x)
        x = self.initial_max_pool(x)
        x = self.initial_bn(x)
        x = self.initial_relu(x)
        x = self.initial_drop_out(x)
        
        for layer in self.intermediate_conv_layers:
            x = layer(x)
            
        return Flatten()(x)
            
        
        
        
        

In [16]:
cnn_tower = CNNTower(num_conv_layers = 3,
                     initial_channel_size = 128,
                     initial_filter_size = (3, 3),
                     dropout_rate = 0.20
                    )
cnn_tower(tf.random.uniform((32, 640, 360, 3), dtype=tf.float32, minval=0, maxval=256))

<tf.Tensor: shape=(32, 448), dtype=float32, numpy=
array([[481.68216 , 117.24413 , 286.0591  , ..., 780.04736 , 229.16895 ,
        431.8591  ],
       [512.6173  , 124.067314, 266.7232  , ..., 797.6442  , 219.39418 ,
        415.07407 ],
       [548.1993  , 140.68118 , 297.74728 , ..., 781.6818  , 202.63403 ,
        422.85928 ],
       ...,
       [562.4627  , 153.69173 , 283.8444  , ..., 767.04944 , 205.20067 ,
        450.4334  ],
       [542.22076 , 173.38083 , 277.2708  , ..., 782.0031  , 201.2753  ,
        441.12256 ],
       [508.33386 ,  80.07395 , 294.59195 , ..., 766.78107 , 221.42638 ,
        430.23126 ]], dtype=float32)>

In [17]:
class DNN(tf.keras.layers.Layer):
    def __init__(self, num_layers: int, initial_dense_size: int, dropout_rate: float):
        self.layers = []
        
        for idx in range(num_layers):
            self.layers.append(Dense(initial_dense_size//(2**idx)))
            self.layers.append(tf.keras.layers.BatchNormalization())
            self.layers.append(LeakyReLU(alpha=0.1))
            
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)

        return x
            
            
    

In [18]:
dnn = DNN(3, 256, 0.2)
dnn(tf.random.uniform((32, 488), dtype=tf.float32))

<tf.Tensor: shape=(32, 64), dtype=float32, numpy=
array([[ 2.44961873e-01, -4.28758897e-02, -5.47311865e-02, ...,
        -2.03106739e-02,  1.53838217e-01,  4.16802168e-01],
       [-2.77533326e-02, -3.77547625e-03, -6.68615028e-02, ...,
        -4.14037853e-02,  4.03724074e-01,  4.00969267e-01],
       [ 4.60152812e-02, -4.00812551e-02, -6.56707808e-02, ...,
        -2.26197410e-02,  1.12396866e-01,  3.81510973e-01],
       ...,
       [-1.29463989e-02, -2.12700125e-02, -1.73682496e-02, ...,
        -1.98369753e-02,  3.35701078e-01,  1.53319106e-01],
       [ 1.87113687e-01, -1.92494951e-02, -5.21184579e-02, ...,
        -1.29027234e-04, -3.17534991e-02,  5.28111517e-01],
       [ 2.07876340e-01, -4.46629897e-02, -5.29783666e-02, ...,
        -4.77412753e-02,  1.16185442e-01,  5.46506226e-01]], dtype=float32)>

# Two Tower Input Model

In [19]:
image_input = Input(shape=IMAGE_EXAMPLE_SHAPE)
cnn_tower = CNNTower(num_conv_layers = 3,
                     initial_channel_size = 128,
                     initial_filter_size = (3, 3),
                     dropout_rate = 0.20
                    )
x_cnn_tower = cnn_tower(image_input)

In [20]:
landmark_input = Input(shape=LANDMARK_EXAMPLE_SHAPE)
dnn_tower = DNN(3, 1024, 0.2)

x_dnn_tower = dnn_tower(landmark_input)

In [21]:
x = concatenate([x_cnn_tower, x_dnn_tower])

In [22]:
output_tower = DNN(3, 256, 0.2)
x = output_tower(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

In [23]:
model = Model(inputs=[image_input, landmark_input], outputs=[output])

In [24]:
opt = optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [25]:
# model.summary()

In [26]:
epochs = 50
batch_size = 32

In [27]:
model.fit([X_train_imgs, X_train_landmarks], Y_train,
          batch_size=batch_size,
          epochs=epochs,
#           callbacks=callbacks,
          verbose=1,
          validation_data=([X_val_imgs, X_val_landmarks], Y_val),
          shuffle=True)

Epoch 1/100
19/19 [==============================] - 9s 451ms/step - loss: 1.1365 - accuracy: 0.5569 - val_loss: 3.6664 - val_accuracy: 0.6700
Epoch 2/100
19/19 [==============================] - 8s 401ms/step - loss: 0.6948 - accuracy: 0.7742 - val_loss: 4.4622 - val_accuracy: 0.6800
Epoch 3/100
19/19 [==============================] - 8s 407ms/step - loss: 0.4847 - accuracy: 0.8234 - val_loss: 3.0830 - val_accuracy: 0.6800
Epoch 4/100
19/19 [==============================] - 8s 411ms/step - loss: 0.3905 - accuracy: 0.8659 - val_loss: 3.0708 - val_accuracy: 0.6800
Epoch 5/100
19/19 [==============================] - 8s 408ms/step - loss: 0.3400 - accuracy: 0.8812 - val_loss: 1.4034 - val_accuracy: 0.6900
Epoch 6/100
19/19 [==============================] - 8s 402ms/step - loss: 0.3013 - accuracy: 0.8947 - val_loss: 1.0876 - val_accuracy: 0.6900
Epoch 7/100
19/19 [==============================] - 8s 399ms/step - loss: 0.2870 - accuracy: 0.9015 - val_loss: 0.8729 - val_accuracy: 0.7400

In [28]:
model.evaluate([X_test_imgs, X_test_landmarks], Y_test, verbose=1)

7/7 [==============================] - 1s 154ms/step - loss: 0.8015 - accuracy: 0.7450


[0.8015316128730774, 0.7450000047683716]

In [29]:
Y_preds = model.predict([X_test_imgs, X_test_landmarks])

In [30]:
Y_test_labels = np.argmax(Y_test, axis=-1)
Y_preds_labels = np.argmax(Y_preds, axis=-1)

In [31]:
print(sklearn.metrics.classification_report(Y_test_labels, Y_preds_labels))

              precision    recall  f1-score   support

           0       0.75      0.92      0.83       133
           1       0.43      0.19      0.26        16
           2       0.85      0.58      0.69        19
           3       0.71      0.31      0.43        16
           4       0.73      0.50      0.59        16

    accuracy                           0.74       200
   macro avg       0.69      0.50      0.56       200
weighted avg       0.73      0.74      0.72       200



# [Ablation] Image only

In [32]:
image_input = Input(shape=IMAGE_EXAMPLE_SHAPE)
cnn_tower = CNNTower(num_conv_layers = 3,
                     initial_channel_size = 128,
                     initial_filter_size = (3, 3),
                     dropout_rate = 0.20
                    )
x_cnn_tower = cnn_tower(image_input)

In [33]:
output_tower = DNN(3, 256, 0.2)
x = output_tower(x_cnn_tower)
output = Dense(NUM_CLASSES, activation='softmax')(x)

In [34]:
ablation_model_image = Model(inputs=[image_input], outputs=[output])

In [35]:
opt = optimizers.Adam()

ablation_model_image.compile(loss='categorical_crossentropy',
                             optimizer=opt,
                             metrics=['accuracy'])

In [36]:
epochs = 50
batch_size = 32

In [37]:
ablation_model_image.fit([X_train_imgs], Y_train,
                         batch_size=batch_size,
                         epochs=epochs,
#                          callbacks=callbacks,
                         verbose=1,
                         validation_data=([X_val_imgs], Y_val),
                         shuffle=True)

Epoch 1/100
19/19 [==============================] - 8s 410ms/step - loss: 1.3634 - accuracy: 0.4635 - val_loss: 10.4732 - val_accuracy: 0.6800
Epoch 2/100
19/19 [==============================] - 8s 400ms/step - loss: 0.9062 - accuracy: 0.6672 - val_loss: 7.8789 - val_accuracy: 0.6800
Epoch 3/100
19/19 [==============================] - 8s 400ms/step - loss: 0.7202 - accuracy: 0.7284 - val_loss: 3.7017 - val_accuracy: 0.6800
Epoch 4/100
19/19 [==============================] - 8s 401ms/step - loss: 0.5950 - accuracy: 0.7606 - val_loss: 3.6300 - val_accuracy: 0.6800
Epoch 5/100
19/19 [==============================] - 8s 400ms/step - loss: 0.5094 - accuracy: 0.7929 - val_loss: 2.4685 - val_accuracy: 0.6700
Epoch 6/100
19/19 [==============================] - 8s 401ms/step - loss: 0.4560 - accuracy: 0.8183 - val_loss: 1.4474 - val_accuracy: 0.6900
Epoch 7/100
19/19 [==============================] - 8s 399ms/step - loss: 0.4958 - accuracy: 0.8048 - val_loss: 0.8938 - val_accuracy: 0.760

In [38]:
Y_preds = ablation_model_image.predict([X_test_imgs, X_test_landmarks])

In [39]:
Y_test_labels = np.argmax(Y_test, axis=-1)
Y_preds_labels = np.argmax(Y_preds, axis=-1)

In [40]:
print(sklearn.metrics.classification_report(Y_test_labels, Y_preds_labels))

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       133
           1       0.24      0.69      0.35        16
           2       0.57      0.42      0.48        19
           3       0.40      0.25      0.31        16
           4       0.50      0.25      0.33        16

    accuracy                           0.59       200
   macro avg       0.49      0.46      0.44       200
weighted avg       0.64      0.59      0.60       200



# [Ablation] Pose Landmarks Only

In [41]:
landmark_input = Input(shape=LANDMARK_EXAMPLE_SHAPE)
dnn_tower = DNN(3, 1024, 0.2)

x_dnn_tower = dnn_tower(landmark_input)

In [42]:
output_tower = DNN(3, 256, 0.2)
x = output_tower(x_dnn_tower)
output = Dense(NUM_CLASSES, activation='softmax')(x)

In [43]:
ablation_model_landmark = Model(inputs=[landmark_input], outputs=[output])

In [44]:
opt = optimizers.Adam()

ablation_model_landmark.compile(loss='categorical_crossentropy',
                                optimizer=opt,
                                metrics=['accuracy'])

In [45]:
epochs = 50
batch_size = 32

In [50]:
ablation_model_landmark.fit([X_train_landmarks], Y_train,
                            batch_size=batch_size,
                            epochs=epochs,
#                          callbacks=callbacks,
                            verbose=1,
                            validation_data=([X_val_landmarks], Y_val),
                            shuffle=True)

Epoch 1/100
19/19 [==============================] - 0s 7ms/step - loss: 0.1155 - accuracy: 0.9576 - val_loss: 0.4120 - val_accuracy: 0.8700
Epoch 2/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1003 - accuracy: 0.9576 - val_loss: 0.4823 - val_accuracy: 0.7900
Epoch 3/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1064 - accuracy: 0.9576 - val_loss: 0.5559 - val_accuracy: 0.8200
Epoch 4/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1289 - accuracy: 0.9491 - val_loss: 1.6369 - val_accuracy: 0.7100
Epoch 5/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1564 - accuracy: 0.9474 - val_loss: 0.7265 - val_accuracy: 0.8200
Epoch 6/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1045 - accuracy: 0.9508 - val_loss: 0.7855 - val_accuracy: 0.7900
Epoch 7/100
19/19 [==============================] - 0s 7ms/step - loss: 0.1055 - accuracy: 0.9508 - val_loss: 0.2318 - val_accuracy: 0.9100
Epoch 8/100
1

In [55]:
Y_preds = ablation_model_landmark.predict([X_test_landmarks])

In [56]:
Y_test_labels = np.argmax(Y_test, axis=-1)
Y_preds_labels = np.argmax(Y_preds, axis=-1)

In [57]:
print(sklearn.metrics.classification_report(Y_test_labels, Y_preds_labels))

              precision    recall  f1-score   support

           0       0.75      0.88      0.81       133
           1       0.30      0.19      0.23        16
           2       0.29      0.11      0.15        19
           3       0.78      0.44      0.56        16
           4       0.89      1.00      0.94        16

    accuracy                           0.73       200
   macro avg       0.60      0.52      0.54       200
weighted avg       0.68      0.72      0.69       200

